# Pembersihan Data Penjualan Harian untuk Analisis Time Series



## **Langkah 1: Memuat dan Memeriksa Data Awal**

Langkah pertama adalah memuat dataset `mock_kaggle.csv` menggunakan library Pandas. Setelah itu, kita akan melakukan inspeksi awal untuk memahami struktur data, tipe kolom, dan melihat beberapa baris pertama. 

In [1]:
import pandas as pd
import numpy as np

# Muat dataset dari file CSV
file_path = 'raw_data.csv'
df = pd.read_csv(file_path)

# Tampilkan 5 baris pertama untuk melihat sampel data
print("Lima Baris Pertama Data Awal:")
display(df.head())

# Tampilkan informasi umum tentang DataFrame
print("\nInformasi Awal DataFrame:")
df.info()

Lima Baris Pertama Data Awal:


,data;venda;estoque;preco
0,01/01/2014;0;4972;1.29
1,02/01/2014;70;4902;1.29
2,03/01/2014;59;4843;1.29
3,04/01/2014;93;4750;1.29
4,05/01/2014;96;4654;1.29



Informasi Awal DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937 entries, 0 to 936
Data columns (total 1 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   data;venda;estoque;preco  937 non-null    object
dtypes: object(1)
memory usage: 7.4+ KB


### **Temuan Awal:**

  * Dataset terdiri dari **937 baris** dan 4 kolom: `data`, `venda`, `estoque`, dan `preco`.
  * Tidak ada nilai yang hilang (non-null) pada data mentah.
  * Kolom `data` (tanggal) saat ini berjenis `object` (teks), yang perlu diubah menjadi `datetime` untuk analisis berbasis waktu.
  * Nama kolom masih dalam Bahasa Portugis/Spanyol. Nantinya akan kita ubah ke Bahasa Inggris untuk standardisasi.

## **Langkah 2: Pembersihan dan Pra-pemrosesan Data**

Pada tahap ini, kita akan melakukan serangkaian transformasi untuk membuat data siap dianalisis.

### **2.1. Konversi Tipe Data Tanggal dan Penyesuaian Indeks**

Langkah paling krusial dalam analisis time series adalah memastikan data memiliki indeks waktu yang benar. Kita akan mengonversi kolom `data` menjadi `datetime` dan menjadikannya sebagai indeks DataFrame.

In [38]:
# Pecah kolom tunggal menjadi beberapa kolom
df = df.iloc[:, 0].str.split(';', expand=True)
df.columns = ['data', 'venda', 'estoque', 'preco']

# Konversi tipe data
df['data'] = pd.to_datetime(df['data'], dayfirst=True, errors='coerce')
df['venda'] = pd.to_numeric(df['venda'], errors='coerce')
df['estoque'] = pd.to_numeric(df['estoque'], errors='coerce')
# Jika desimal pakai koma, ganti koma dulu:
df['preco'] = pd.to_numeric(df['preco'].str.replace(',', '.'), errors='coerce')

# Set index waktu dan urutkan
df = df.set_index('data').sort_index()

print("Tipe data indeks setelah konversi:", df.index.dtype)
display(df.head())

Tipe data indeks setelah konversi: datetime64[ns]


,venda,estoque,preco
data,,,
2014-01-01,0,4972,1.29
2014-01-02,70,4902,1.29
2014-01-03,59,4843,1.29
2014-01-04,93,4750,1.29
2014-01-05,96,4654,1.29


### **2.2. Mengidentifikasi dan Mengisi Tanggal yang Hilang**

Analisis time series yang andal memerlukan data dengan interval yang konsisten (dalam kasus ini, harian). Kita akan memeriksa apakah ada "lubang" atau tanggal yang datanya tidak tercatat.


In [41]:
# Buat rentang tanggal harian yang lengkap dari tanggal min hingga max
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

# Temukan tanggal yang ada di rentang lengkap tapi tidak ada di data kita
missing_dates = full_range.difference(df.index)

print(f"Ditemukan {len(missing_dates)} tanggal yang hilang dalam data.")
if not missing_dates.empty:
    print("Contoh tanggal yang hilang:", missing_dates[:5].strftime('%Y-%m-%d').tolist())

# Lakukan re-index untuk menambahkan baris baru untuk tanggal yang hilang
# Baris baru ini akan berisi nilai NaN (Not a Number)
df = df.reindex(full_range)

print("\nJumlah nilai kosong (NaN) setelah re-indexing:")
print(df.isnull().sum())

Ditemukan 6 tanggal yang hilang dalam data.
Contoh tanggal yang hilang: ['2014-01-17', '2014-02-10', '2014-02-11', '2014-03-04', '2014-03-15']

Jumlah nilai kosong (NaN) setelah re-indexing:
venda      6
estoque    6
preco      6
dtype: int64


### **2.3. Imputasi Data (Mengisi Nilai yang Hilang)**

Setelah menambahkan baris untuk tanggal yang hilang, kita perlu mengisi nilai kosong (`NaN`) tersebut dengan strategi yang logis (imputasi).

  * **`venda` (Penjualan)**: Akan diisi dengan **0**. Asumsinya, jika tidak ada catatan, maka tidak ada penjualan.
  * **`estoque` (Stok) & `preco` (Harga)**: Akan diisi dengan nilai dari hari sebelumnya **(Forward Fill)**. Asumsinya, stok dan harga tidak berubah jika tidak ada aktivitas penjulan.


In [44]:
# Imputasi untuk kolom 'venda'
df['venda'] = df['venda'].fillna(0)

# Imputasi untuk kolom 'estoque' dan 'preco' menggunakan forward fill
# ffill() akan membawa nilai valid terakhir untuk mengisi NaN
df['estoque'] = df['estoque'].ffill()
df['preco'] = df['preco'].ffill()

# Cek kembali apakah masih ada nilai yang kosong
print("Jumlah nilai kosong setelah imputasi:")
print(df.isnull().sum())

Jumlah nilai kosong setelah imputasi:
venda      0
estoque    0
preco      0
dtype: int64


### **2.4. Koreksi Anomali Data**

Kita perlu memastikan tidak ada nilai yang tidak logis dalam data, contohnya harga yang bernilai nol atau negati



In [47]:
# Temukan baris di mana harga <= 0
invalid_prices = df[df['preco'] <= 0]
print(f"Ditemukan {len(invalid_prices)} baris dengan harga tidak valid (<= 0).")

# Ganti harga tidak valid dengan NaN, lalu lakukan ffill lagi
df.loc[df['preco'] <= 0, 'preco'] = np.nan
df['preco'] = df['preco'].ffill()

print("\nJumlah harga tidak valid setelah koreksi:", (df['preco'] <= 0).sum())

Ditemukan 10 baris dengan harga tidak valid (<= 0).

Jumlah harga tidak valid setelah koreksi: 0


## **Langkah 3: Finalisasi dan Ekspor Data**

Sebagai langkah terakhir, kita akan mengubah nama kolom ke Bahasa Inggris untuk standardisasi dan menyimpan hasilnya ke dalam file CSV baru.

In [58]:
# Buat salinan DataFrame yang sudah bersih
df_cleaned = df.copy()

# Ganti nama kolom ke Bahasa Inggris
df_cleaned = df_cleaned.rename(columns={
    'venda': 'sales',
    'estoque': 'stock',
    'preco': 'price'
})

# Tampilkan informasi akhir dan sampel data
print("Informasi Akhir DataFrame:")
df_cleaned.info()

print("\nLima Baris Pertama Data Bersih:")
display(df_cleaned.head())

# Simpan DataFrame bersih ke file CSV baru
output_path = 'cleaned_dataset_en.csv'
# Pastikan header untuk index (tanggal) muncul sebagai 'date'
df_cleaned.to_csv(output_path, index_label='date')

print(f"\nData yang sudah bersih berhasil diekspor ke: {output_path}")

Informasi Akhir DataFrame:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 943 entries, 2014-01-01 to 2016-07-31
Freq: D
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sales   943 non-null    float64
 1   stock   943 non-null    float64
 2   price   943 non-null    float64
dtypes: float64(3)
memory usage: 61.8 KB

Lima Baris Pertama Data Bersih:


,sales,stock,price
2014-01-01,0.0,4972.0,1.29
2014-01-02,70.0,4902.0,1.29
2014-01-03,59.0,4843.0,1.29
2014-01-04,93.0,4750.0,1.29
2014-01-05,96.0,4654.0,1.29



Data yang sudah bersih berhasil diekspor ke: cleaned_dataset_en.csv


Proses pembersihan data telah selesai. Dataset awal yang memiliki **937 baris** dengan beberapa tanggal yang hilang kini telah menjadi dataset yang lengkap dan berurutan sebanyak **943 baris**. Seluruh nilai yang tidak logis telah dikoreksi dan format data telah distandarisasi. Dataset ini sekarang **siap dan andal** untuk digunakan dalam tahap analisis dan peramalan selanjutnya.